In [1]:
using Distributions, CSV, DataFrames, ProgressMeter
include("../../../utils_sg.jl")

### Parameters

In [2]:
T = 1.0; r=0.05; S0 = 100.0; v0 = 0.4; α = 0.4; β = 0.9; ρ = 0.3; K = 100.0;
slvm = SABRSLV(S0::Float64, v0::Float64, α::Float64, β::Float64, ρ::Float64, r::Float64);
Π = [1.0 ρ; ρ 1.0]; cholΠ=cholesky(Π).L

drift(x) = [slvm.ω(abs(x[1]),abs(x[2])), slvm.μ(abs(x[2]))]
diffusion(x) = Diagonal(
        [slvm.m(abs(x[2]))*slvm.Γ(abs(x[1])), slvm.σ(abs(x[2]))]
        )*cholΠ

R = r;
driver(t, x, y, z) = (
    -r*y
)
terminal(x) = max(x[1]-K,0)
bsde = BSDE(T, [S0, v0], drift, diffusion, driver, terminal);

### Hagan's formula

In [3]:
function implied_volatility(t::Float64, f::Float64, v::Float64)
    # hagan
    if f != K
        numerator = (
            v * (
                1 + (
                    (1-β)^2/24 * v^2/(f*K)^(1-β) 
                    + 1/4 * ρ*β*α*v/(f*K)^((1-β)/2)
                    + (2-3*ρ^2)/24 * α^2
                    ) * t
                )
            )
        denominator = (
            (f*K)^((1-β)/2) * (
                1 + (1-β)^2/24 * log(f/K)^2
                + (1-β)^4/1920 * log(f/K)^4
                )
            )
        z = α/v * (f*K)^((1-β)/2) * log(f/K)
        x = log((sqrt(1-2*ρ*z+z^2)+z-ρ)/(1-ρ))
        return numerator/denominator * z/x
    else
        numerator = v * (
            1 + (
                (1-β)^2/24 * v^2/f^(2-2*β) 
                + 1/4 * ρ*β*α*v/f^(1-β)
                + (2-3*ρ^2)/24 * α^2
                ) * t
        )
        denominator = f^(1-β)
        return numerator/denominator
    end
end;

function BSvanilla(volatility, strike, expiry, spot, interest_rate, dividend_rate)
    forward = spot#*exp((interest_rate-dividend_rate)*expiry)
    sqrt_var = volatility * sqrt(expiry)
    if sqrt_var > 0.0
        d1 = log(forward/strike)/sqrt_var + sqrt_var/2
        d2 = d1 - sqrt_var
        call = forward*cdf(Normal(), d1) - strike*cdf(Normal(), d2)
        # put = -forward*cdf(Normal(), -d1) + exp(-(interest_rate-dividend_rate)*expiry)*strike*cdf(Normal(), -d2)
    else
        call = max(forward-strike, 0.0)
        # put = max(strike-forward, 0.0)
    end
    return exp(-(interest_rate-dividend_rate)*expiry)*call #put
end;

function price(t::Float64, f::Float64, v::Float64)
    return BSvanilla(implied_volatility(T-t, f, v), K, T-t, f, r, 0.0)
end;


## Experiment

In [4]:
Nₜs = [10, 20, 50, 100, 200, 500]
header = vcat(["q", "measurement_type"], string.(Nₜs))
scheme = HochOst4(krylov=true, m=100); EXPINT = true;

In [5]:
### designing grids
d = 2
### Parameters for Tavella-Randall grids
g = 5.0
g₁=g
g₂=g

lefts = [0.0, 0.0];
rights = [200.0, 0.8];
centers = [100.0, 0.4];
scaled_centers=(centers.-(rights.+lefts)./2.0)./((rights.-lefts)./2.0);

In [6]:
function solveProb(mind, bsde, Nₜ, scheme)
    d = length(mind)
    
    _p = [rescale(eq_nodes(mind[dim]), scaled_centers[dim], lefts[dim], rights[dim]) for dim in 1:d]
    Δₗ = [pp[2]-pp[1] for pp in _p]
    Δᵣ = [pp[end]-pp[end-1] for pp in _p]
    Nₗ = [length(pp)÷2 for pp in _p]
    Nᵣ = [length(pp)÷2 for pp in _p]

    ratioₗ = Δₗ.*Nₗ./(Δₗ[1]*Nₗ[1])
    ratioᵣ = Δᵣ.*Nᵣ./(Δᵣ[1]*Nᵣ[1])
    grids = Array{AbstractGrid,1}(undef, d)
    grids[1] = TavellaRandallGrid(g₁, g₂, _p[1][1], _p[1][length(_p[1])÷2+1], _p[1][end], length(_p[1])÷2, length(_p[1])÷2)
    grids[2] = Grid1D(_p[2], length(_p[2]), length(_p[2])÷2, length(_p[2])÷2)
    res = MethodOfLines(bsde, grids, Nₜ, scheme, true)
    return [res[1], grids]
end

solveProb (generic function with 1 method)

In [7]:
eval_grids = Array{AbstractGrid,1}(undef, 2)
eval_grids[1] = begin
    tr = TavellaRandallGrid(g₁, g₂, lefts[1], centers[1], rights[1], 100, 100)
    gr = tr.grid[findall(attr->(attr<1.2*bsde.X0[1])&&(attr>0.8*bsde.X0[1]), tr.grid)]
    Grid1D(gr, length(gr), length(gr[gr.<centers[1]]), length(gr[gr.>centers[1]]))
end
eval_grids[2] = begin
    reg = Grid1D(lefts[2]:(centers[2]/15):rights[2],31,15,15)
    gr = reg.grid[findall(attr->(attr<1.2*bsde.X0[2])&&(attr>0.8*bsde.X0[2]), reg.grid)]
    Grid1D(gr, length(gr), length(gr[gr.<centers[1]]), length(gr[gr.>centers[1]]))
end

Grid1D([0.3466666666666667, 0.37333333333333335, 0.4, 0.4266666666666667, 0.4533333333333333], 5, 5, 0)

In [8]:
qs = [7,8,9,10]
schemename = split(split(string(scheme), '{')[1], '(')[1]
df = DataFrame([[],[],[],[],[],[],[],[]], header)
sol100 = zeros(length(qs), length(Nₜs))
solsup = zeros(length(qs), length(Nₜs))
runtimes = zeros(length(qs), length(Nₜs))
@showprogress for (ind, Nₜ) in enumerate(Nₜs)
    print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
    correct = [price(T-(time-1)/Nₜ, (p,v)...) for time in 1:Nₜ+1, p in eval_grids[1].grid, v in eval_grids[2].grid]
    @showprogress for (qind,q) in enumerate(qs)
        print(string("q=", q,":\n")); flush(stdout)
        spind = sparse_indices(q, d)
        exc_start = time()
        results = [solveProb(mind, bsde, Nₜ, scheme) for mind in spind]
        data = [dat[1] for dat in results]
        grids = [dat[2] for dat in results]
        if all([size(dat)[2] == Nₜ+1 for dat in data])
            sg_interp = SparseGridInterpolation(data, grids, spind, q, d, Nₜ, lefts, rights, centers)
            exc_stop = time()
            sol100[qind, ind] = abs.(price(0.0, [100.0,0.4]...)-sg_interp.Interpolant(Nₜ+1, [100.0,0.4]))
            result = [sg_interp.Interpolant(time, [p,v]) for time in 1:Nₜ+1, p in eval_grids[1].grid, v in eval_grids[2].grid]
            solsup[qind, ind] = maximum(abs.(result-correct))
        else
            exc_stop = time()
            sol100[qind,ind] = Inf
            solsup[qind,ind] = Inf
        end
        runtimes[qind,ind] = exc_stop - exc_start
        print(string("Sup Err=", solsup[qind,ind],":\n")); flush(stdout)
        print(string("Y0(100,0.4)=", sol100[qind,ind],":\n")); flush(stdout)
        print(string("Runtime[s]=", runtimes[qind,ind],":\n")); flush(stdout)
    end
end

for (qind,q) in enumerate(qs)
    row_sup = vcat([string(q), "Sup Err"], string.(solsup[qind,:])); push!(df,row_sup);
    row_100 = vcat([string(q), "Y0(100,0.4)"], string.(sol100[qind,:])); push!(df,row_100);
    row_run = vcat([string(q), "Runtime[s]"], string.(runtimes[qind,:])); push!(df,row_run);
end
df |> CSV.write(string(schemename, ".csv"))

Nₜ=10:
q=7:
Sup Err=0.04521204462466777:
Y0(100,0.4)=0.006920885565280699:
Runtime[s]=10.149618148803711:
q=8:
Sup Err=0.03469262459509981:
Y0(100,0.4)=0.0024542363635138287:
Runtime[s]=2.547924041748047:


Progress:  50%|████████████████████▌                    |  ETA: 0:00:13

q=9:
Sup Err=0.39611024233798453:
Y0(100,0.4)=0.11932070553846863:
Runtime[s]=4.611959934234619:


Progress:  75%|██████████████████████████████▊          |  ETA: 0:00:06

q=10:
Sup Err=5.537168266459178:
Y0(100,0.4)=4.941641806461591:
Runtime[s]=12.31826400756836:


...

q=9:
Sup Err=0.0322946798007111:
Y0(100,0.4)=0.0013083637832274064:
Runtime[s]=116.59639120101929:


Progress:  75%|██████████████████████████████▊          |  ETA: 0:01:08

q=10:
Sup Err=0.03164570032920633:
Y0(100,0.4)=0.0010321589664421538:
Runtime[s]=283.5138330459595:


Progress: 100%|█████████████████████████████████████████| Time: 0:08:09
Progress: 100%|█████████████████████████████████████████| Time: 0:15:22


"HochOst4.csv"